## Légifrance data

### Preprocess data 

In [1]:
from pylegifrance import LegiHandler
import config2py

client = LegiHandler()
client.set_api_keys(legifrance_api_key=config2py.config_getter("LEGIFRANCE_CLIENT_ID"), legifrance_api_secret=config2py.config_getter("LEGIFRANCE_CLIENT_SECRET"))

2024-02-21 10:41:04,705 - root - INFO - Connexion à l'api legifrance réussie.


In [2]:
from pylegifrance import recherche_CODE

doc_integrale_travail = recherche_CODE(code_name="Code du travail")

2024-02-21 10:41:12,986 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1


In [3]:
len(doc_integrale_travail[0])

41

In [5]:
doc_integrale_travail[0].keys()

dict_keys(['executionTime', 'dereferenced', 'id', 'idConteneur', 'cid', 'title', 'nor', 'eli', 'alias', 'jorfText', 'jurisState', 'visa', 'modifDate', 'jurisDate', 'dateDebutVersion', 'dateFinVersion', 'signers', 'prepWork', 'dateParution', 'dateTexte', 'numParution', 'notice', 'nota', 'inap', 'textNumber', 'textAbroge', 'etat', 'dossiersLegislatifs', 'nature', 'resume', 'rectificatif', 'motsCles', 'appellations', 'liens', 'observations', 'sections', 'articles', 'pagePdf', 'fileName', 'fileSize', 'filePath'])

In [6]:
## Voir le contenu de doc_integrale
import json

with open('code_travail.json', 'w', encoding='utf-8') as fichier:
    json.dump(doc_integrale_travail, fichier, ensure_ascii=False, indent=4)

In [7]:
def extraire_infos(element, infos=[]):
    if isinstance(element, dict):
        if 'pathTitle' in element and 'content' in element:
            infos.append({
                'pathTitle': element['pathTitle'],
                'content': element['content']
            })

        for key, value in element.items():
            extraire_infos(value, infos)

    elif isinstance(element, list):
        for item in element:
            extraire_infos(item, infos)

    return infos

In [8]:
infos_extraites_code_travail = extraire_infos(doc_integrale_travail[0]['sections'])

In [9]:
len(infos_extraites_code_travail)

11520

In [10]:
for info in infos_extraites_code_travail[:5]:
    print(info)

{'pathTitle': ['Partie réglementaire  ', 'Première partie : Les relations individuelles de travail ', 'Livre Ier : Dispositions préliminaires ', "Titre Ier : Champ d'application et calcul des seuils d'effectifs", ' Chapitre unique'], 'content': "<p>En application de l'article <a href='/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000006900783&dateTexte=&categorieLien=cid'>L. 1111-2</a>, les salariés mis à disposition par une entreprise de travail temporaire, un groupement d'employeurs ou une association intermédiaire ne sont pas pris en compte pour le calcul des effectifs de l'entreprise utilisatrice pour l'application des dispositions légales relatives à la tarification des risques accident du travail et maladie professionnelle qui se réfèrent à une condition d'effectif.</p>"}
{'pathTitle': ['Partie réglementaire  ', 'Première partie : Les relations individuelles de travail ', 'Livre Ier : Dispositions préliminaires ', 'Titre IV : Egalité professionnelle entre 

In [11]:
## pour sos oxygène, c'est que les données de santé, sécurité sociale, ....
code_sante = recherche_CODE(code_name="Code de la santé publique")

2024-02-21 10:42:26,727 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1


In [12]:
infos_extraites_code_sante = extraire_infos(code_sante[0]['sections'])
infos_extraites_code_sante[:2]

[{'pathTitle': ['Partie réglementaire  ',
   'Première partie : Les relations individuelles de travail ',
   'Livre Ier : Dispositions préliminaires ',
   "Titre Ier : Champ d'application et calcul des seuils d'effectifs",
   ' Chapitre unique'],
  'content': "<p>En application de l'article <a href='/affichCodeArticle.do?cidTexte=LEGITEXT000006072050&idArticle=LEGIARTI000006900783&dateTexte=&categorieLien=cid'>L. 1111-2</a>, les salariés mis à disposition par une entreprise de travail temporaire, un groupement d'employeurs ou une association intermédiaire ne sont pas pris en compte pour le calcul des effectifs de l'entreprise utilisatrice pour l'application des dispositions légales relatives à la tarification des risques accident du travail et maladie professionnelle qui se réfèrent à une condition d'effectif.</p>"},
 {'pathTitle': ['Partie réglementaire  ',
   'Première partie : Les relations individuelles de travail ',
   'Livre Ier : Dispositions préliminaires ',
   'Titre IV : Egal

In [13]:
len(infos_extraites_code_sante)

25819

In [14]:
with open('code_sante.json', 'w', encoding='utf-8') as fichier:
    json.dump(infos_extraites_code_sante, fichier, ensure_ascii=False, indent=4)

## Create dict with content in text format

In [15]:
infos_extraites_code_sante[0].keys()

dict_keys(['pathTitle', 'content'])

In [16]:
## The content is in html format, so I convert it to text using utf8 encoding and I ignore links as they have not a meaning

import html2text

def convert_content(content, ig_links=True):
    h = html2text.HTML2Text()
    h.ignore_links = ig_links
    if isinstance(content, bytes):
        content = content.decode('utf-8')
    return h.handle(content)

converted_infos = []
id_counter = 1 

for item in infos_extraites_code_sante:
    converted_item = {
        'pathTitle': item['pathTitle'],
        'content': convert_content(item['content']),
        'id': f'id{id_counter}' 
    }



    converted_infos.append(converted_item)
    id_counter += 1

In [17]:
len(converted_infos)

25819

## Chunker (using lkj chunker)

In [18]:
from typing import Iterable
from lkj import chunker
import oa


def pathTitle_list_to_text(title: list, sep='\n'):
    return sep.join(title)

def pathContent_metadata_to_text(metadata: dict, *, sep='\n'):
    return f"{sep.join(metadata['pathTitle'])}{sep}{metadata['content']}"

In [19]:
from typing import List, Tuple
import tiktoken

def tokenize(text: str, encoding_name : str ) -> List[str]:
    encoding = tiktoken.get_encoding(encoding_name)
    int_tokens = encoding.encode(text)
    str_tokens = [encoding.decode_single_token_bytes(token) for token in int_tokens]
    return str_tokens

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

For the moment, I will skip code that exceeds the maximum token limit

In [20]:
from lkj import chunker
encoding_name = "cl100k_base" 

def process_data(data: dict, max_tokens: int) -> Tuple[str, dict, bool]:
    code_text = pathContent_metadata_to_text(data)
    code_tokens = num_tokens_from_string(code_text, encoding_name)

    if code_tokens > max_tokens:
        # code_tokens = code_tokens[:max_tokens]
        return "", {}, True  # Skips the code

    # truncated_text = ' '.join(code_tokens)
    # return truncated_text
    metadata = {
        "source": pathTitle_list_to_text(data['pathTitle']),
        "id" : data['id'],
        
    }
    return code_text, metadata, False

In [117]:
## Embedding one batch
max_tokens = 8192
batch_size =  20
single_batch = next(chunker(converted_infos, batch_size))
texts = [code_text for code_text, _, skip in map(lambda code: process_data(code, max_tokens), single_batch) if not skip]
embeddings = oa.embeddings(texts)

2024-02-21 09:59:59,270 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## ChromaDB (our vector db)

In [28]:
#chroma run --path /Users/Sana/chromadb   

import chromadb
from chromadb.config import Settings

import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key= config2py.config_getter("OPENAI_API_KEY"),
                model_name="text-embedding-3-small"
            )
chroma_client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.token.TokenAuthClientProvider",
                      chroma_client_auth_credentials="test-token"))
chroma_client.heartbeat()  
code_collection = chroma_client.get_or_create_collection(name="rag_sante", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)
chroma_client.list_collections() 
## To delete: chroma_client.delete_collection(name="rag_sante")

2024-02-21 10:54:44,629 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-02-21 10:54:44,679 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


[Collection(name=rag_sante), Collection(name=rag_jokes)]

In [30]:
max_tokens = 8192
batch_size = 20
code_chunker = chunker(converted_infos, batch_size)  

for single_batch in code_chunker:
    processed_batch = map(lambda code: process_data(code, max_tokens), single_batch)
    
    valid_chunks = [(code_text, metadata) for code_text, metadata, skip in processed_batch if not skip]

    if valid_chunks:
        texts, metadatas = zip(*valid_chunks)
        ids = [metadata['id'] for metadata in metadatas]

        # embeddings = oa.embeddings(list(texts))

        # for embedding, code_text, code_id, metadata in zip(embeddings, texts, ids, metadatas):
        #     code_collection.add(embeddings=embedding, documents=code_text, ids=code_id, metadatas=metadata)
        for code_text, code_id, metadata in zip(texts, ids, metadatas):
            code_collection.add(documents=code_text, ids=code_id, metadatas=metadata)

2024-02-21 10:55:16,268 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:25,821 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:26,371 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:26,888 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:27,334 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:27,661 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:28,031 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:28,677 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-21 10:58:28,983 - httpx - INFO - HTTP Request: POST https://api.openai.c

KeyboardInterrupt: 

In [ ]:
# code_collection.get(include=['embeddings', 'documents', 'metadatas'])